In [1]:
import torch
import trimesh
import numpy as np

def spherical_to_cartesian(theta, phi, device="cpu"):
    """Convert spherical coordinates (azimuth theta, elevation phi) to Cartesian coordinates."""
    return torch.tensor([torch.sin(phi) * torch.cos(theta),
                         torch.sin(phi) * torch.sin(theta),
                         torch.cos(phi)], device=device,dtype=torch.float32)

def calculate_projected_area(normal, area, view_vector):
    """Calculate the projected area of a triangle for a given view vector."""
    dot_product = torch.dot(normal, view_vector)  # Scalar dot product
    return area * torch.abs(dot_product)

def calculate_angular_change_with_periodicity(theta1, phi1, theta2, phi2):
    """Calculate the angular change between two spherical angles, considering periodicity."""
    # Convert spherical coordinates into unit vectors using sin and cos for periodicity
    x1, y1 = torch.cos(theta1), torch.sin(theta1)
    x2, y2 = torch.cos(theta2), torch.sin(theta2)
    # Compute angular differences considering periodicity for both azimuth and elevation
    theta_diff = torch.acos(torch.clamp(x1 * x2 + y1 * y2, -1.0, 1.0))
    # For elevation (phi), it's simpler as we can treat it like the azimuth
    phi_diff = torch.abs(phi1 - phi2)
    phi_diff = torch.minimum(phi_diff, 2 * np.pi - phi_diff)  # Wrap around periodicity for phi

    return theta_diff, phi_diff

def calculate_similarity(mesh, view1, view2, weight_area=0.5, weight_normals=0.5, device="cpu"):
    """Calculate the similarity between two views based on projected areas and normal vector changes."""
    # Convert spherical to cartesian coordinates for both views
    view_vector1 = spherical_to_cartesian(torch.tensor(view1[0], device=device), torch.tensor(view1[1], device=device), device=device)
    view_vector2 = spherical_to_cartesian(torch.tensor(view2[0], device=device), torch.tensor(view2[1], device=device), device=device)

    # Convert face normals and areas to PyTorch tensors and move them to GPU if necessary
    normals = torch.tensor(mesh.face_normals, dtype=torch.float32, device=device)  # (num_faces, 3)
    areas = torch.tensor(mesh.area_faces, dtype=torch.float32, device=device)  # (num_faces,)

    # Initialize arrays for projected areas and angular changes
    projected_areas_similarity = []
    area_view1 = 0.
    area_view2 = 0.
    # Iterate through all faces (triangles)
    for face_idx in range(len(mesh.faces)):
        normal = normals[face_idx]  # Normal of the face
        area = areas[face_idx]  # Area of the face

        # Calculate projected areas for both views 就是算了个投影面积
        projected_area1 = calculate_projected_area(normal, area, view_vector1)
        projected_area2 = calculate_projected_area(normal, area, view_vector2)
        if torch.dot(normal, view_vector1) < 0: #如果面法向量normal和方向矢量view_vector1成钝角则算入投影面积
            area_view1 += projected_area1
        if torch.dot(normal, view_vector2) < 0:
            area_view2 += projected_area2
    projected_areas_difference = torch.abs((area_view1-area_view2))
    similarity = 1 / (1 + 10*projected_areas_difference.item())
    similarity = max(0, min(similarity, 1)) #确保在0-1
    dissim = 1-similarity
    return similarity, dissim
    # return projected_areas_difference


# Example usage
device = 'cuda:0'
# Load your STL file (adjust path accordingly)
filename = 'planes/b7fd11d4af74b4ffddaa0161e9d3dfac.obj'
mesh = trimesh.load(filename)  # Use trimesh to load the mesh

##Example view angles (azimuth and elevation in degrees)
# views = [(30, 45, 30, 45), (30, 45, 60, 45), (30, 45, 90, 45), (30, 45, 120, 45), (30, 45, 150, 45)]
# for view in views:

viewls = []
for i in range(20):
    viewls.append((np.random.randint(0,180),np.random.randint(0,360),np.random.randint(0,180),np.random.randint(0,360)))
for view in viewls:

    view1 = (np.radians(view[0]), np.radians(view[1]))
    view2 = (np.radians(view[2]), np.radians(view[3]))
    
    # Calculate the similarity between the two views using both projected area and normal
    similarity, dissim= calculate_similarity(mesh, view1, view2, weight_area=0.5, weight_normals=0., device=device) #想起来应该有周期性，所以可以把单独的角度分量去掉
    print(f"Area Similarity between {view} (theta phi theta phi): {similarity:.4f}, dissim:{dissim:.4f}")
'''output
Difference between (30, 45, 30, 45) (theta phi theta phi): 0.0000
Difference between (30, 45, 60, 45) (theta phi theta phi): 0.0276
Difference between (30, 45, 90, 45) (theta phi theta phi): 0.0379
Difference between (30, 45, 120, 45) (theta phi theta phi): 0.0262
Difference between (30, 45, 150, 45) (theta phi theta phi): 0.0022
beta=10
Area Similarity between (30, 45, 30, 45) (theta phi theta phi): 1.0000
Area Similarity between (30, 45, 60, 45) (theta phi theta phi): 0.7839
Area Similarity between (30, 45, 90, 45) (theta phi theta phi): 0.7249
Area Similarity between (30, 45, 120, 45) (theta phi theta phi): 0.7925
Area Similarity between (30, 45, 150, 45) (theta phi theta phi): 0.9782

'''
'''
Area Difference between (3, 225, 100, 51) (theta phi theta phi): 0.0668
Area Difference between (113, 352, 115, 66) (theta phi theta phi): 0.0449
Area Difference between (79, 123, 35, 120) (theta phi theta phi): 0.0402
Area Difference between (92, 101, 1, 209) (theta phi theta phi): 0.0737
Area Difference between (95, 93, 95, 11) (theta phi theta phi): 0.0574
Area Difference between (106, 36, 63, 107) (theta phi theta phi): 0.0172
Area Difference between (150, 352, 80, 19) (theta phi theta phi): 0.0159
Area Difference between (140, 176, 69, 164) (theta phi theta phi): 0.0112
Area Difference between (167, 212, 122, 310) (theta phi theta phi): 0.0394
Area Difference between (126, 277, 105, 349) (theta phi theta phi): 0.0316
Area Difference between (120, 246, 27, 143) (theta phi theta phi): 0.0394
Area Difference between (27, 137, 28, 64) (theta phi theta phi): 0.0019
Area Difference between (115, 299, 51, 198) (theta phi theta phi): 0.0357
Area Difference between (74, 292, 77, 201) (theta phi theta phi): 0.0386
Area Difference between (148, 71, 107, 349) (theta phi theta phi): 0.0028
Area Difference between (123, 53, 50, 59) (theta phi theta phi): 0.0028
Area Difference between (170, 4, 175, 63) (theta phi theta phi): 0.0304
Area Difference between (117, 111, 54, 119) (theta phi theta phi): 0.0097
Area Difference between (175, 309, 132, 2) (theta phi theta phi): 0.0219
Area Difference between (104, 150, 75, 164) (theta phi theta phi): 0.0146
'''

Area Similarity between (13, 20, 84, 213) (theta phi theta phi): 0.7429, dissim:0.2571
Area Similarity between (67, 354, 92, 90) (theta phi theta phi): 0.6117, dissim:0.3883
Area Similarity between (129, 16, 13, 285) (theta phi theta phi): 0.7368, dissim:0.2632
Area Similarity between (139, 169, 91, 27) (theta phi theta phi): 0.8108, dissim:0.1892
Area Similarity between (1, 316, 13, 237) (theta phi theta phi): 0.9876, dissim:0.0124
Area Similarity between (87, 307, 25, 277) (theta phi theta phi): 0.6286, dissim:0.3714
Area Similarity between (38, 159, 104, 10) (theta phi theta phi): 0.9918, dissim:0.0082
Area Similarity between (114, 191, 31, 252) (theta phi theta phi): 0.9886, dissim:0.0114
Area Similarity between (143, 84, 98, 249) (theta phi theta phi): 0.6682, dissim:0.3318
Area Similarity between (83, 329, 122, 326) (theta phi theta phi): 0.9414, dissim:0.0586
Area Similarity between (135, 282, 81, 120) (theta phi theta phi): 0.7637, dissim:0.2363
Area Similarity between (155, 19

'\nArea Difference between (3, 225, 100, 51) (theta phi theta phi): 0.0668\nArea Difference between (113, 352, 115, 66) (theta phi theta phi): 0.0449\nArea Difference between (79, 123, 35, 120) (theta phi theta phi): 0.0402\nArea Difference between (92, 101, 1, 209) (theta phi theta phi): 0.0737\nArea Difference between (95, 93, 95, 11) (theta phi theta phi): 0.0574\nArea Difference between (106, 36, 63, 107) (theta phi theta phi): 0.0172\nArea Difference between (150, 352, 80, 19) (theta phi theta phi): 0.0159\nArea Difference between (140, 176, 69, 164) (theta phi theta phi): 0.0112\nArea Difference between (167, 212, 122, 310) (theta phi theta phi): 0.0394\nArea Difference between (126, 277, 105, 349) (theta phi theta phi): 0.0316\nArea Difference between (120, 246, 27, 143) (theta phi theta phi): 0.0394\nArea Difference between (27, 137, 28, 64) (theta phi theta phi): 0.0019\nArea Difference between (115, 299, 51, 198) (theta phi theta phi): 0.0357\nArea Difference between (74, 292